In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import pickle
from pathlib import Path


In [ ]:


# ---------- ECDF utils ----------
def build_ecdf_model(values: np.ndarray) -> dict:
    v = np.asarray(values)
    v = v[~np.isnan(v)]
    if v.size < 10:
        raise ValueError(f"Not enough samples to build ECDF: {v.size}")
    x_sorted = np.sort(v)
    N = x_sorted.size
    eps = 1.0 / (N + 1)  # keep p in (0,1)
    return {"x_sorted": x_sorted, "N": N, "eps": eps}


def ecdf_p(x: np.ndarray, model: dict, clip: bool = True) -> np.ndarray:
    xs = model["x_sorted"]
    N = model["N"]
    p = np.searchsorted(xs, x, side="right") / (N + 1)
    if clip:
        eps = model["eps"]
        p = np.clip(p, eps, 1 - eps)
    return p


# ---------- Window -> ECDF -> p -> SREPI ----------
def window_ecdf_srepi_one_site(
    df: pd.DataFrame,
    value_col: str,
    window: str,
    time_col: str = "time",
    agg: str = "mean",               
    min_periods: int | None = None,   
    keep_original: bool = True
) -> tuple[pd.DataFrame, dict]:
    """
    对单站点：
    1) time-based rolling window 聚合 value_col
    2) 对 window 序列构建 ECDF
    3) 映射 p
    4) 计算 SREPI = norm.ppf(p)
    """
    d = df.copy()
    d[time_col] = pd.to_datetime(d[time_col])
    d = d.sort_values(time_col).set_index(time_col)

    expected = int(pd.Timedelta(window).total_seconds() // 3600)

    if min_periods is None:
        min_periods = expected

    if agg == "mean":
        w_series = d[value_col].rolling(window=window, min_periods=min_periods).mean()
    elif agg == "sum":
        w_series = d[value_col].rolling(window=window, min_periods=min_periods).sum()
    else:
        raise ValueError("agg must be 'mean' or 'sum'")

    w_col = f"{value_col}_w{window}"
    out = pd.DataFrame({w_col: w_series}).dropna().reset_index()

    if keep_original:
        out = out.merge(
            d[[value_col]].reset_index(),
            on=time_col,
            how="left"
        )

    model = build_ecdf_model(out[w_col].to_numpy())

    p_col = f"p_w{window}"
    srepi_col = f"srepi_w{window}"
    out[p_col] = ecdf_p(out[w_col].to_numpy(), model, clip=True)
    out[srepi_col] = norm.ppf(out[p_col].to_numpy())

    return out, model


def window_ecdf_srepi_all_sites(
    data_dict: dict,
    value_col: str,
    windows: list[str],
    time_col: str = "time",
    agg: str = "mean",
    min_periods_map: dict[str, int] | None = None
) -> tuple[dict, dict]:
    """
    批量：{site: df(time,value)} -> 每个 window 各自产出 {site: out_df} + {site: ecdf_model}
    """
    results_by_window = {w: {} for w in windows}
    models_by_window  = {w: {} for w in windows}

    for site, df_site in data_dict.items():
        for w in windows:
            mp = None if min_periods_map is None else min_periods_map.get(w, None)
            out_df, model = window_ecdf_srepi_one_site(
                df=df_site,
                value_col=value_col,
                window=w,
                time_col=time_col,
                agg=agg,
                min_periods=mp,
                keep_original=True
            )
            results_by_window[w][site] = out_df
            models_by_window[w][site] = model

    return results_by_window, models_by_window


WINDOWS = ["1h", "4h", "12h", "24h", "48h", "72h", "120h"]

In [9]:
with open("../data/processed/solar_ecdf_model.pkl", "rb") as f:
    ecdf_models = pickle.load(f)

ecdf_solar = {}
for window in WINDOWS:
    dir_solar_ecdf = Path(f"../data/processed/solar_ecdf/{window}")
    files_solar_ecdf = list(dir_solar_ecdf.glob("*"))
    ecdf_solar_window = {file.stem: pd.read_parquet(file) for file in files_solar_ecdf}
    ecdf_solar[window] = ecdf_solar_window

In [13]:

def flatten_ecdf_solar(
    ecdf_solar: dict,
    window_col="window",
    code_col="code",
    time_col="time",
    sort_by_time=True,
):
    parts = []

    for window, code_dict in ecdf_solar.items():
        for code, df in code_dict.items():
            d = df.copy()
            d[window_col] = window
            d[code_col] = code

            if time_col in d.columns:
                d[time_col] = pd.to_datetime(d[time_col], errors="coerce")
                if sort_by_time:
                    d = d.sort_values(time_col)

            parts.append(d)

    return pd.concat(parts, ignore_index=True)



def add_solar_srepi(df, p_col="p", out_col="solar_srepi", eps=1e-6):
    d = df.copy()
    d["p_clip"] = d[p_col].clip(eps, 1 - eps)
    d[out_col] = norm.ppf(d["p_clip"])
    return d

In [ ]:

df_solar_all = flatten_ecdf_solar(
    ecdf_solar,
    window_col="window",
    code_col="code",
    time_col="time",
    sort_by_time=True,
)

print("Flattened df shape:", df_solar_all.shape)
print(df_solar_all.head())


df_solar_all_srepi = add_solar_srepi(
    df_solar_all,
    p_col="p",
    out_col="solar_srepi",
    eps=1e-6
)

df_solar_all_srepi

In [14]:
df_solar_all_srepi

,time,rsds,rsds_w1h,p,window,code,rsds_w4h,rsds_w12h,rsds_w24h,rsds_w48h,rsds_w72h,rsds_w120h,p_clip,solar_srepi
0,2014-01-01 00:30:00,585.375000,585.375000,0.625939,1h,0CULCSF,NaN,NaN,NaN,NaN,NaN,NaN,0.625939,0.321117
1,2014-01-01 01:30:00,320.296875,320.296875,0.294537,1h,0CULCSF,NaN,NaN,NaN,NaN,NaN,NaN,0.294537,-0.540178
2,2014-01-01 02:30:00,917.109375,917.109375,0.908101,1h,0CULCSF,NaN,NaN,NaN,NaN,NaN,NaN,0.908101,1.329151
3,2014-01-01 03:30:00,745.671875,745.671875,0.777648,1h,0CULCSF,NaN,NaN,NaN,NaN,NaN,NaN,0.777648,0.764275
4,2014-01-01 06:30:00,335.968750,335.968750,0.315678,1h,0CULCSF,NaN,NaN,NaN,NaN,NaN,NaN,0.315678,-0.479820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27781651,2023-12-31 08:30:00,205.500000,NaN,0.650427,120h,YATSF1,NaN,NaN,NaN,NaN,NaN,608.956641,0.650427,0.386474
27781652,2023-12-31 20:30:00,171.109375,NaN,0.643231,120h,YATSF1,NaN,NaN,NaN,NaN,NaN,606.802734,0.643231,0.367109
27781653,2023-12-31 21:30:00,391.000000,NaN,0.633991,120h,YATSF1,NaN,NaN,NaN,NaN,NaN,604.018099,0.633991,0.342443
27781654,2023-12-31 22:30:00,610.906250,NaN,0.621140,120h,YATSF1,NaN,NaN,NaN,NaN,NaN,600.083594,0.621140,0.308475


In [15]:
df_solar_all_srepi.to_parquet("../data/processed/solar_srepi.parquet", index=False)